# Geodesics in $\Gamma$

This notebook provides a proof for the proposition

***Proposition***. The manifolds $N_{i,I}^\pm$ are uniquely tessellated by copies of $P$.

The proof strategy can be described as follows:
- let $T$ be the lift to $\mathbb H^5$ of the standard tessellation of $N_{i,I}^\pm$, as constructed in the paper;
- for concreteness, we can assume that $T$ is the tessellation induced by the canonical representation of the Coxeter group $\Gamma$;
- assume that there exists another tessellation, then it can be described as the projection of $\gamma(T)$ for some $\gamma \in \mathrm{Isom}(\mathbb H^5)$;
- we show that, give a manifold $N_{i,I}^\pm$, we can find some specific _shapes_ in its universal cover $\mathbb H^5$
  (based on geodesics and hyperbolic subspaces), ultimately inducing a configuration of five orthogonal lines;
- we also show that these configurations can only occur in $32$ positions with respect to the tessellation;
- of these $32$, only $8$ are distinct, of which $7$ can be excluded as they do not project to a valid tessellation of $N_{i,I}^\pm$;
- due to these constraints, the tessellation $T$ is unique.

In [1]:
import itertools
import functools
import random
from collections import deque, defaultdict

In [2]:
def multiset(it):
    d = defaultdict(int)
    for k in it:
        d[k] += 1
    return dict(d)

def multimap(it):
    d = defaultdict(set)
    for k,v in it:
        d[k].add(v)
    return dict(d)

In [3]:
def print_matrix(m, digits=3):
    print(m.str(rep_mapping=lambda x: str(x.n(digits=digits))))

In [4]:
def make_coxeter_group(F, edges):
    n = F.ngens()
    all_edges = {(i, j) : 1 if i == j else 2 for i in range(n) for j in range(i,n)}
    for a,b,e in edges:
        all_edges[min(a,b), max(a,b)] = e
    rels = [(F.gen(k[0])*F.gen(k[1]))^v for k,v in all_edges.items() if v >= 0 and v != oo]
    return F/rels

In [5]:
fr.<a,b,c,d,e> = FreeGroup()
cox_edges = [(0,1,5),(1,2,3),(2,3,3),(3,4,3)]
G = make_coxeter_group(fr, cox_edges); G

Finitely presented group < a, b, c, d, e | a^2, (a*b)^5, (a*c)^2, (a*d)^2, (a*e)^2, b^2, (b*c)^3, (b*d)^2, (b*e)^2, c^2, (c*d)^3, (c*e)^2, d^2, (d*e)^3, e^2 >

In [6]:
long_words = [
    a*b*c*d*e,
    b*a*b*c*b*a*b*a*c*b*a*d*c*e*d*c*b*a*b*a*c*d*e*b*c*a*b*a*b*c*d*c*b*a*b*c*a*b*a*b*c,
    b*a*b*c*b*a*d*c*b*a*b*a*c*b*a*d*e*d*c*b*a*b*c*d*e*a*b*a*b*c*d*b*a*b*c*a*b*a*b*c*d*c*b,
    b*c*b*a*b*a*d*c*b*a*b*a*c*b*a*b*c*d*c*b*a*b*a*c*d*b*c*a*b*a*b*c*d*e*b*c*a*b*a*b*c*d*b*a*b*c*a*b,
    b*a*b*a*c*d*c*b*a*b*a*c*b*a*d*c*b*a*b*c*d*e*d*c*b*a*b*c*d*a*b*c*a*b*a*b*c*d*e*b*a*b*c*d*b*c*a*b*a*b*c
]
W = d*b*c*a*e*d*b*c*e*d*e
w1,w2,w3,w4,w5 = [W * w * W^-1 for w in long_words]

The following are generators for the fundamental groups of the Long manifolds. \
In order, they generate the groups $H_i' \coloneqq W \cdot H_i \cdot W^{-1}$, where $H_i$ is one of $H_1, H_4, H_5, H_8, H_2, H_3, H_6, H_7$. \
Conjugation with the word $W$ will be useful later: it allows for each $W \cdot H \cdot W^{-1}$ to contain $(bdace)^2$, \
an isometry whose axis intersects the fundamental chamber $P$ in the canonical representation. \
Note that $(H_1, H_8)$ and $(H_3, H_6)$ are conjugate pairs in $G$.

In [7]:
long_gens = [
    [w2^-2, w3, w4*w2^-1, w4^-1*w2^-1, w2*w1*w2^-1, w2*w3*w2^-1, w1, w5],
    [w1, w2^-2, w3*w2^-1, w3^-1*w2^-1, w4*w2^-1, w4^-1*w2^-1, w5*w2^-1, w5^-1*w2^-1],
    [w1^-2, w2, w3, w4*w1^-1, w1*w2*w1^-1, w1*w3*w1^-1, w5],
    [w1^-2, w3*w1^-1, w3^-1*w1^-1, w2, w4*w1^-1, w5*w1^-1],
    
    [w1, w2^-2, w3, w4*w2^-1, w5*w2^-1, w2*w1*w2^-1, w2*w3*w2^-1, w4^-1*w2^-1, w5^-1*w2^-1],
    [w1, w2^-2, w3*w2^-1, w3^-1*w2^-1, w4*w2^-1, w4^-1*w2^-1, w5],
    [w1^-2, w3, w4*w1^-1, w1*w2*w1^-1, w2, w5*w1^-1, w5^-1*w1^-1],
    
    [w1^-2, w2, w5, w3*w1^-1, w4*w1^-1, w3^-1*w1^-1]
]

In [8]:
CK = G.subgroup(long_words)

In [9]:
SGs = [G.subgroup(long_gen) for long_gen in long_gens]

## Basic definitions

In [10]:
def nf_element(x):
    """Returns the algebraic number x as a number field element."""
    return QQ[x].gen(0)

In [11]:
Qphi.<phi> = NumberField(x^2-x-1, embedding=1.6)
print(phi.n())

1.61803398874989


Next, we define the standard Lorentzian form on $\mathbb R^{5,1}$, `J`,\
and the defining form `Jphi` of the prism $P$.

`Jphi` defines a model of $\mathbb H^5$, which we shall call the _scaled hyperboloid model_:\
$$\mathbb H^5 \simeq \{v \in \mathbb R^{5,1} \mid v^t\cdot \mathtt{Jphi}\cdot v = -\varphi, v_0 > 0\}.$$

In [12]:
J = diagonal_matrix([-1,1,1,1,1,1])
Jphi = diagonal_matrix([-phi,1,1,1,1,1])

In [13]:
def normalize(v, ring = AA):
    """
    Returns a multiple of v lying on the scaled hyperboloid model.
    Assumes that v is already in the positive cone of the model.
    """
    return vector(ring, v / sqrt(-v * Jphi * v / phi), immutable=True)

In [14]:
def cosh_dist(v,w):
    """Returns the hyperbolic cosine of the distance between two points v, w in the scaled hyperboloid model."""
    return -v * Jphi * w / phi

In [15]:
def pyth(a,b):
    """Returns the hypotenuse of a hyperbolic right triangle of sides a, b."""
    return acosh(cosh(a)*cosh(b))

In [16]:
def midpoint(v, w):
    """Returns the midpoint of two points v, w in the scaled hyperboloid model."""
    return normalize(v + w)

The outer normal vectors to each facet:

In [17]:
facet_normals = [vector(Qphi, l, immutable=True) for l in [
    (1, phi^2, 0, 0, 0, 0),
    (0, -1, 1, 0, 0, 0),
    (0, 0, -1, 1, 0, 0),
    (0, 0, 0, -1, 1, 0),
    (0, 0, 0, 0, -1, 1),
    (0, 0, 0, 0, 0, -1),
    (phi, 1, 1, 1, 1, 1),
]]

In [18]:
facet_reflections = [
    matrix(Qphi, 1-2*(u.outer_product(u*Jphi))/(u*Jphi*u), immutable=True) for u in facet_normals
]
facet_reflections

[
[ phi   -1    0    0    0    0]  [1 0 0 0 0 0]  [1 0 0 0 0 0]
[ phi -phi    0    0    0    0]  [0 0 1 0 0 0]  [0 1 0 0 0 0]
[   0    0    1    0    0    0]  [0 1 0 0 0 0]  [0 0 0 1 0 0]
[   0    0    0    1    0    0]  [0 0 0 1 0 0]  [0 0 1 0 0 0]
[   0    0    0    0    1    0]  [0 0 0 0 1 0]  [0 0 0 0 1 0]
[   0    0    0    0    0    1], [0 0 0 0 0 1], [0 0 0 0 0 1],

[1 0 0 0 0 0]  [1 0 0 0 0 0]  [ 1  0  0  0  0  0]
[0 1 0 0 0 0]  [0 1 0 0 0 0]  [ 0  1  0  0  0  0]
[0 0 1 0 0 0]  [0 0 1 0 0 0]  [ 0  0  1  0  0  0]
[0 0 0 0 1 0]  [0 0 0 1 0 0]  [ 0  0  0  1  0  0]
[0 0 0 1 0 0]  [0 0 0 0 0 1]  [ 0  0  0  0  1  0]
[0 0 0 0 0 1], [0 0 0 0 1 0], [ 0  0  0  0  0 -1],

[ 5*phi + 4 -2*phi - 1 -2*phi - 1 -2*phi - 1 -2*phi - 1 -2*phi - 1]
[ 3*phi + 2       -phi   -phi - 1   -phi - 1   -phi - 1   -phi - 1]
[ 3*phi + 2   -phi - 1       -phi   -phi - 1   -phi - 1   -phi - 1]
[ 3*phi + 2   -phi - 1   -phi - 1       -phi   -phi - 1   -phi - 1]
[ 3*phi + 2   -phi - 1   -phi - 1   -phi - 1      

In [19]:
facet_hyperplanes = [
    (m-1).right_kernel() for m in facet_reflections
]

Facets incident to each of the 10 vertices of $P$,
where $0,1,\dots,6$ stand for $a,b,\dots,g$.

In [20]:
vertices_facets = [
    (0,1,2,3,5),
    (0,1,2,4,5),
    (0,1,3,4,5),
    (0,2,3,4,5),
    (1,2,3,4,5),
    (0,1,2,3,6),
    (0,1,2,4,6),
    (0,1,3,4,6),
    (0,2,3,4,6),
    (1,2,3,4,6),
]

In [21]:
def intersect_facets(facet_indices):
    inters = functools.reduce(
        lambda V, W: V.intersection(W),
        (facet_hyperplanes[j] for j in facet_indices)
    ).gen(0)
    return vector(
        AA, normalize(inters), immutable = True
    )

In [22]:
vertices = [
    intersect_facets(fs)
    for fs in vertices_facets
]

In [23]:
vertices

[(4.236067977499789?, 2.618033988749895?, 2.618033988749895?, 2.618033988749895?, 2.618033988749895?, 0),
 (1.851229586821916?, 1.144122805635369?, 1.144122805635369?, 1.144122805635369?, 0, 0),
 (1.376381920471174?, 0.8506508083520399?, 0.8506508083520399?, 0, 0, 0),
 (1.144122805635369?, 0.7071067811865475?, 0, 0, 0, 0),
 (1, 0, 0, 0, 0, 0),
 (4.846581979279201?, 2.995352392457285?, 2.995352392457285?, 2.995352392457285?, 2.995352392457285?, 0.7071067811865475?),
 (2.995352392457285?, 1.851229586821916?, 1.851229586821916?, 1.851229586821916?, 1.144122805635369?, 1.144122805635369?),
 (2.727546913047446?, 1.685716698173176?, 1.685716698173176?, 1.256459042640573?, 1.256459042640573?, 1.256459042640573?),
 (2.618033988749895?, 1.618033988749895?, 1.309016994374948?, 1.309016994374948?, 1.309016994374948?, 1.309016994374948?),
 (2.558336368008464?, 1.339562313220224?, 1.339562313220224?, 1.339562313220224?, 1.339562313220224?, 1.339562313220224?)]

Checking that `facet_normals` are indeed _outer_ normals:

In [24]:
[sum(vertices) * Jphi * rv < 0 for rv in facet_normals]

[True, True, True, True, True, True, True]

A special point of $P$:

In [25]:
mid = midpoint(vertices[4], vertices[5])

In [26]:
[cosh_dist(mid, v) for v in vertices]

[1.57337286723525?,
 1.30698302899770?,
 1.43225365432768?,
 1.57337286723526?,
 1.709763430899024?,
 1.70976343089902?,
 1.34913003104219?,
 1.31727239286769?,
 1.30698302899770?,
 1.30215779520651?]

The point `mid` has maximum distance $\cosh^{-1}(1.70976\dots)$ from a vertex of $P$. \
By convexity, a ball of this radius centered on `mid` contains $P$.

In [27]:
cosh_radius = cosh_dist(mid, vertices[4])

In [28]:
def axis(m, l):
    """Returns the axis of a hyperbolic isometry m, assuming that it has an eigenvalue l with |l| != 1."""
    return ((m - l) * (m - 1/l)).right_kernel()

In [29]:
def orthogonal_projection(bm):
    """Given a subspace S spanned by the columns of bm, returns the orthogonal projection matrix onto S.""" 
    return bm * (bm.transpose() * Jphi * bm)^-1 * bm.transpose() * Jphi

In [30]:
def double_projection(p1, p2):
    """Returns the composition of two projections, first onto p2, then onto p1"""
    return orthogonal_projection(p1.basis_matrix().transpose()) * orthogonal_projection(p2.basis_matrix().transpose())

In [31]:
def cosh_dist_hyperbolic_flats(p1, p2):
    """
    Returns the hyperbolic cosine of the minimum distance
    between two hyperbolic subspaces of the scaled hyperboloid model.
    """
    dp = double_projection(p1, p2)
    return sqrt(max((x for x in dp.eigenvalues() if x > 1), default = 1))

In [32]:
def cos_angle_hyperbolic_flats(p1, p2):
    """
    Returns the cosine of the angle between two intersecting
    hyperbolic subspaces of the scaled hyperboloid model.
    """
    dp = double_projection(p1, p2)
    return sqrt(min((abs(x) for x in dp.eigenvalues() if 0 < abs(x) < 1), default = 1))

In [33]:
def free_word(w):
    char_to_gen = dict(zip("abcde", fr.gens()))
    return product(char_to_gen[ch] for ch in w)

In [34]:
def G_matrix(w):
    if not w or w == fr.one():
        return identity_matrix(Jphi, 6)
    if isinstance(w, str):
        w = free_word(w)
    return w(facet_reflections[:5])

## Possible eigenvalues of hyperbolic isometries

> **_Note:_** This section may be skipped; it is dedicated to computing possible eigenvalues of hyperbolic isometries, \
which may be useful to classify geodesics of $\Gamma$ of various lengths.

The largest eigenvalue of a hyperbolic isometry defined over $\mathbb Q(\sqrt 5)$ is a positive algebraic integer $\lambda$.
Its conjugates are $1/\lambda$ and some unit complex numbers. \
Hence, $\lambda$ is one of:
1. a unit of the ring of integers of $\mathbb Q(\sqrt 5)$ (if it has no unit norm conjugates);
2. a Salem number.

Moreover, if we consider isometries of $\mathbb R^{5,1}$, the degree of $\lambda$ over $\mathbb Q$ is at most $12$.

Lists of small Salem numbers can be found at <http://wayback.cecm.sfu.ca/~mjm/Lehmer/lists/>, complete up to $2.2$. \
To them we should add the powers of the golden ratio $\varphi$, to cover case 1. However, already $\varphi^2 > 2.2$. Hence, we only add $\varphi$.

In [38]:
import urllib.request

In [39]:
def fetch_salem_numbers(url):
    sl = []
    with urllib.request.urlopen(url) as txt:
        for line in txt:
            deg, value, poly = line.decode().split()
            sl.append((float(value), QQ[x](sage_eval(poly, locals={"z":x}))))
    return sl

In [40]:
%time salem_numbers = sum([fetch_salem_numbers(f"http://wayback.cecm.sfu.ca/~mjm/Lehmer/lists/S{deg}.txt") for deg in (4,6,8,10,12)], [])

CPU times: user 205 ms, sys: 1.58 ms, total: 207 ms
Wall time: 2.58 s


In [41]:
eigenvalues = salem_numbers + [(phi.n(), QQ[x](x^2-x-1))] # add phi to the list

In [42]:
eigenvalues.sort()

In [43]:
eigenvalues[:10]

[(1.17628081825992, x^10 + x^9 - x^7 - x^6 - x^5 - x^4 - x^3 + x + 1),
 (1.21639166113827, x^10 - x^6 - x^5 - x^4 + 1),
 (1.23039143440722, x^10 - x^7 - x^5 - x^3 + 1),
 (1.24072642365254, x^12 - x^11 + x^10 - x^9 - x^6 - x^3 + x^2 - x + 1),
 (1.26123096113714, x^10 - x^8 - x^5 - x^2 + 1),
 (1.28063815626776, x^8 - x^5 - x^4 - x^3 + 1),
 (1.29348595312545, x^10 - x^8 - x^7 + x^5 - x^3 - x^2 + 1),
 (1.30226880509433, x^12 - x^11 - x^7 + x^6 - x^5 - x + 1),
 (1.31591443192595, x^12 - x^8 - x^7 - x^6 - x^5 - x^4 + 1),
 (1.33731321020199, x^10 - x^9 - x^5 - x + 1)]

Next, we remove all possible eigenvalues $>2.2$, or of degree $>6$ whose number field is not an extension of $\mathbb Q(\sqrt 5)$.

In [44]:
%%time
eigenvalues_filtered = [t for t in eigenvalues
                        if t[0] < 2.201
                        and not (t[1].degree() > 6 and QQ[x](x^2-x-1).change_ring(NumberField(t[1], "a")).is_irreducible())]

CPU times: user 2.33 s, sys: 1.78 ms, total: 2.34 s
Wall time: 2.35 s


In [45]:
eigenvalues_filtered

[(1.3599997117115, x^8 - x^7 + x^6 - 2*x^5 + x^4 - 2*x^3 + x^2 - x + 1),
 (1.40126836793985, x^6 - x^4 - x^3 - x^2 + 1),
 (1.41810868701378,
  x^12 - 2*x^11 + 2*x^10 - 2*x^9 + x^8 - x^7 + x^6 - x^5 + x^4 - 2*x^3 + 2*x^2 - 2*x + 1),
 (1.48856532775194,
  x^12 + x^11 - x^9 - 2*x^8 - 3*x^7 - 3*x^6 - 3*x^5 - 2*x^4 - x^3 + x + 1),
 (1.50613567955384, x^6 - x^5 - x^3 - x + 1),
 (1.54719696562639,
  x^8 - 2*x^7 + 2*x^6 - 3*x^5 + 3*x^4 - 3*x^3 + 2*x^2 - 2*x + 1),
 (1.55603019132268, x^6 - x^5 - x^4 + x^3 - x^2 - x + 1),
 (1.58234718368046, x^6 - x^4 - 2*x^3 - x^2 + 1),
 (1.61803398874989, x^2 - x - 1),
 (1.63557312992222, x^6 - 2*x^5 + 2*x^4 - 3*x^3 + 2*x^2 - 2*x + 1),
 (1.64859385517863,
  x^12 - 2*x^11 + x^10 - x^9 + x^8 - 2*x^7 + 3*x^6 - 2*x^5 + x^4 - x^3 + x^2 - 2*x + 1),
 (1.69207764384217,
  x^12 - 3*x^11 + 4*x^10 - 5*x^9 + 6*x^8 - 7*x^7 + 7*x^6 - 7*x^5 + 6*x^4 - 5*x^3 + 4*x^2 - 3*x + 1),
 (1.72208380573905, x^4 - x^3 - x^2 - x + 1),
 (1.72757204869731,
  x^12 - x^10 - x^9 - 2*x^8 - 2*x^

In [46]:
lambda_fields = [QQ[AA.polynomial_root(ZZ[x](p), RIF(1,2.5))] for v,p in eigenvalues_filtered]
lambdas       = [fld.gen(0) for fld in lambda_fields]
lambdas_rif   = [RIF(l) for l in lambdas]

for j,l in enumerate(lambdas):
    print(f"{j:>2}: {l.n()}")

 0: 1.35999971171150
 1: 1.40126836793985
 2: 1.41810868701378
 3: 1.48856532775194
 4: 1.50613567955384
 5: 1.54719696562639
 6: 1.55603019132268
 7: 1.58234718368046
 8: 1.61803398874989
 9: 1.63557312992222
10: 1.64859385517863
11: 1.69207764384217
12: 1.72208380573904
13: 1.72757204869729
14: 1.73336396362049
15: 1.78164359860800
16: 1.80017173933249
17: 1.83107582510231
18: 1.84959921585537
19: 1.88320350591353
20: 1.91649826367085
21: 1.94685626827188
22: 1.95530180508213
23: 1.96355303898882
24: 1.97481870829771
25: 1.98779316684622
26: 2.01103224818394
27: 2.01854676887552
28: 2.02441384179332
29: 2.04249053394077
30: 2.06149844119604
31: 2.08101899662454
32: 2.12971030981632
33: 2.15372137554177
34: 2.19564673649222


## A special eigenvalue

In [35]:
# Qlam, lam = lambda_fields[18], lambdas[18]
Qlam.<lam> = NumberField(x^8+x^7-x^6-4*x^5-5*x^4-4*x^3-x^2+x+1, embedding=1.85)
print(lam.n())

1.84959921585537


In [36]:
lam_rif = RIF(lam)

## Tessellation search

In [37]:
def cosh_dist_bound(x, rad):
    """
    Bounds the translation distance of a certain point under a hyperbolic isometry g
    whose axis intersects P, given:
        - x:    exp of translation distance of g
        - rad:  cosh of the radius of a ball centered on the point
                and containing P
    Return type:
        - real interval with 53 bits of precision
    Return value:
        - cosh of an upper bound on the translation distance
    """
    return RIF((x+1)^2/(2*x) * rad^2 - 1)

In [38]:
COSH_MAXD = cosh_dist_bound(lam, cosh_radius)
COSH_MAXD, COSH_MAXD.absolute_diameter()

(5.416999196353712?, 8.88178419700125e-16)

In [39]:
def tessellation_search(cosh_maxd, pivot = None, string = False):
    """
    Performs a breadth-first search on the dual graph of the tessellation
    of H^5 into copies of P.
        - cosh_maxd:   cosh of maximum translation
        - pivot:       a normalized point in P, or None for the centroid "mid"
        - string:      whether to also yield the group element as a word
    Yield type: tuple
    Yield value:
        - an exact matrix
        - a RIF matrix
        - (if string) a list of integers
    """
    if pivot is None:
        pivot = mid
    pivot_rif = pivot.change_ring(RIF)
    cosh_maxd_phi = cosh_maxd * RIF(phi)
    I6 = matrix(identity_matrix(Qphi, 6), immutable=True)
    seen = {I6}
    q = deque()
    
    q.append((I6, I6.change_ring(RIF), []) if string else (I6, I6.change_ring(RIF)))
    
    refl_rif = [r.change_ring(RIF) for r in facet_reflections]
    J_rif = Jphi.change_ring(RIF)
    idx = 0
    s = None

    while q:
        if idx % 100 == 0:
            print(idx, end="\r", flush=True)
            
        tup = q.popleft()
        yield tup
        if string:
            m, m_rif, s = tup
        else:
            m, m_rif = tup
            
        for j,(r,r_rif) in enumerate(zip(facet_reflections, refl_rif)):
            mm = m * r
            mm.set_immutable()
            if mm in seen:
                continue
            mm_rif = m_rif * r_rif
            v = mm_rif * pivot_rif
            if -pivot_rif * J_rif * v > cosh_maxd_phi:
                continue
            q.append((mm,mm_rif,s+[j]) if string else (mm,mm_rif))
            seen.add(mm)
        idx += 1
    print()

In [40]:
def is_hyperbolic(m):
    """Every hyperbolic element has largest eigenvalue > 4/3."""
    return det(4/3 - m) < 0

In [41]:
def has_eigenvalue(m, eigv):
    return det(eigv - m) == 0

In [42]:
def has_eigenvalue_rif(m, eigv):
    return det(eigv - m).contains_zero()

We now search for hyperbolic isometries of translation length up to $\log(1.849599\dots)$, corresponding to `COSH_MAXD`.

In [43]:
%time geodesics_lam = [m0 for m0, m1 in tessellation_search(COSH_MAXD) if has_eigenvalue_rif(m1, lam_rif)]

2592300
CPU times: user 50min 4s, sys: 15.6 s, total: 50min 20s
Wall time: 50min 24s


In [44]:
len(geodesics_lam)

22080

The main search, performed for geodesics of length up to $2.153721\dots$
(note the `bounds[-1]`.)

Code:
```
%%time
mats_len = [m0 for m0,m1 in bfs_norepr_exact_iterator(bounds[-1]) if check_pos_eigval_exact(m0)]
```
Output:
```
3306500
CPU times: user 1h 8min 10s, sys: 1min 46s, total: 1h 9min 57s
Wall time: 1h 10min 3s
```

Code:
```
%time mats_len2 = [m for m in mats_len if det(9/4-m) > 0]
```
Output:
```
CPU times: user 7min 3s, sys: 32.3 s, total: 7min 35s
Wall time: 7min 36s
```

We have `(len(mats_len), len(mats_len2)) == (2878944, 293640)`.

Saved as:
```
with open("geodesics-2_15372.dat", "wb") as fil:
    pickle.dump(mats_len2, fil)
```

## Representatives for isometries up to conjugacy

In [43]:
def facet_prods(pt):
    return [pt * Jphi * fn for fn in facet_normals]

def facet_signs(pt):
    return [sign(pr) for pr in facet_prods(pt)]

def inside_fundamental_chamber(pt):
    """Checks whether pt is in the *closed* fundamental chamber."""
    return all(pr <= 0 for pr in facet_prods(pt))

Jphi_fn_rif = [(Jphi * fn).change_ring(RIF) for fn in facet_normals]
def outside_fundamental_chamber_rif(pt):
    pt_rif = pt.change_ring(RIF)
    return any(pt * v > 0 for v in Jphi_fn_rif)
    
print(inside_fundamental_chamber(sum(vertices)))

True


In [44]:
def conjugate_axis_intersecting_P(m, eigv = None, axis_coeff = 2, pivot_phi = None, silent=True):
    """
    Returns a conjugate of the hyperbolic isometry m
    whose axis intersects P in a nontrivial segment.
        - eigv:        largest eigenvalue of m (optional)
        - axis_coeff:  used to choose a point q on the axis (default: 2)
        - pivot_phi:   a point of P whose distance from q is minimized
                       (default: sum of vertices of P)
    Return type:
        - tuple[matrix, list[int]]
    Return value:
        - a conjugate matrix m
        - a list of indices, corresponding to a word in G: the conjugating element
    """
    if eigv is None:
        print(max(m.eigenvalues()))
        lam = nf_element(max(m.eigenvalues()))
    else:
        lam = eigv
    k1 = (m-lam).right_kernel().gen(0).n()
    k2 = (m-1/lam).right_kernel().gen(0).n()
    if k1[0] < 0:
        k1 = -k1
    if k2[0] < 0:
        k2 = -k2
        
    axis_pt = k1 + k2*axis_coeff
    axis_pt = normalize(axis_pt, RDF)
    
    if pivot_phi is None:
        bary_phi = normalize(sum(vertices).n(), RDF)
    else:
        bary_phi = pivot_phi
    
    def dist(p):
        return acosh(-p * Jphi * bary_phi), p[0] #tuple(acosh(-p * Jphi * verts_phi[j]).n() for j in (0,1,2,3,4,5))
    dst = dist(axis_pt)
    word = []
    EPSILON = 1e-7
    while True:
        cur = -1
        if not silent:
            print()
            print()
            print(dst)
            print(facet_signs(axis_pt))
        for k,r in enumerate(facet_reflections):
            if word and k == word[-1]:
                continue
            mr = r * m * r
            axis_ptr = r * axis_pt
            dst2 = dist(axis_ptr)
            if not silent:
                print(f"{k} {dst2[0]:.5f} {dst[1]:.2f}", end = "\t")
            if dst2[0] < dst[0] - EPSILON or dst2[0] < dst[0] + EPSILON and dst2[1] < dst[1] - EPSILON:
                dst = dst2
                newm = mr
                new_axis_pt = axis_ptr
                cur = k
        if cur != -1:
            word.append(cur)
            m = newm
            axis_pt = new_axis_pt
        else:
            if not silent:
                print()
                print(f"Final point on the axis: {axis_pt} @ distance {dst}")
                print(f"Final signs: {fc_prods(axis_pt)}")
            m.set_immutable()
            return (m, word[::-1])

In [45]:
def axis_segment_endpoints(ax, silent=True):
    """
    Given a 2-dimensional subspace ax of the scaled hyperboloid model,
    intersecting the fundamental chamber P in a nontrivial segment,
    returns the two endpoints of this segment.
    """
    inters = [fp.change_ring(ax.base_ring()).intersection(ax) for fp in facet_hyperplanes]
    if not silent:
        print("Dimensions of intersections:", [sp.dimension() for sp in inters])
    
    endpoints = set()
    for sp in inters:
        if sp.dimension() != 1:
            continue
        pt = sp.gen(0)
        pt.set_immutable()
        if inside_fundamental_chamber(pt):
            endpoints.add(pt)
            
    if len(endpoints) != 2:
        #print(matrix([facet_signs(sp.gen(0)) for sp in inters]))
        print("Dimensions of intersections:", [sp.dimension() for sp in inters])
        raise ValueError(f"Found {len(endpoints)} endpoints instead of 2")
    return sorted(endpoints)

In [46]:
def composite_reflection(pt, silent=True):
    supp_facets = [pt * Jphi * fac == 0 for fac in facet_normals]
    if not silent:
        print("".join("abcdefg"[j] for j,ch in enumerate(supp_facets) if ch))
    face = functools.reduce(lambda a,b: a&b, (pl for pl, cond in zip(facet_hyperplanes, supp_facets) if cond))
    bm = face.basis_matrix().transpose()
    proj = bm * (bm.transpose() * Jphi * bm)^-1 * bm.transpose() * Jphi
    return 1 - 2 * proj

In [47]:
@functools.cache
def standardize_intersecting_P(m, eigv, return_points = False, silent = True):
    ax = axis(m, eigv)
    mats = [m]
    pts = axis_segment_endpoints(ax, silent)
    while len(pts) <= 2 or pts[-2:] != pts[:2]:
        cref = composite_reflection(pts[-1], silent)
        if not silent:
            print("---cref---")
            print(cref)
            print()
        m = cref * m * cref
        m.set_immutable()
        mats.append(m)
        pts2 = axis_segment_endpoints(axis(m, eigv), silent)
        if not silent:
            print(f"> {pts2[0].change_ring(RDF)}\n> {pts2[1].change_ring(RDF)}")
        xn = [p for p in pts2 if p != pts[-1]]
        assert(len(xn) == 1)
        if not silent:
            print(xn[0].change_ring(RDF))
        pts.append(xn[0])
        
        
        if not silent:
            for pt in pts:
                print(f">> {pt.change_ring(RDF)}")
                
    min_mat = min(mats)
    min_mat.set_immutable()
    return (min_mat, tuple(pts[:-2])) if return_points else min_mat

In [48]:
def conjugacy_representative(m, eigv = None, return_points = False, silent = True):
    if eigv is None:
        lam = nf_element(max(m.eigenvalues()))
    else:
        lam = eigv
    m, word = conjugate_axis_intersecting_P(m, lam, silent=silent)
    return standardize_intersecting_P(m, lam, return_points=return_points, silent=silent)

In [51]:
%%time
geodesics_repr = set()
for j, gl in enumerate(geodesics_lam):
    geodesics_repr.add(conjugacy_representative(gl, lam))
    print(f"Done {j+1:>4} out of {len(geodesics_lam)}", end="\r", flush=True)
print()

Done 22080 out of 22080
CPU times: user 31min 18s, sys: 14.4 s, total: 31min 33s
Wall time: 31min 11s


In [52]:
len(geodesics_repr)

2

In [53]:
[(m.det(), m) for m in sorted(geodesics_repr)]

[(
   [phi + 1    -phi       0      -1       0       0]
   [      0       0       0       0       0       1]
   [phi + 1    -phi       0    -phi       0       0]
   [      0       0       0       0       1       0]
   [    phi    -phi       0       0       0       0]
1, [      0       0       1       0       0       0]
),
 (
-1,

[ 6*phi + 4 -3*phi - 1 -2*phi - 1 -2*phi - 2 -2*phi - 1 -2*phi - 1]
[ 3*phi + 2   -phi - 1   -phi - 1   -phi - 1   -phi - 1       -phi]
[ 4*phi + 3 -2*phi - 1   -phi - 1 -2*phi - 1   -phi - 1   -phi - 1]
[ 3*phi + 2   -phi - 1   -phi - 1   -phi - 1       -phi   -phi - 1]
[ 4*phi + 2 -2*phi - 1   -phi - 1   -phi - 1   -phi - 1   -phi - 1]
[ 3*phi + 2   -phi - 1       -phi   -phi - 1   -phi - 1   -phi - 1]
)]

Up to conjugacy in $\Gamma$, there is exactly one hyperbolic isometry with largest eigenvalue $1.849599\dots$ and determinant $1$,
corresponding to a _nice geodesic_:

In [54]:
nice_geodesic1 = sorted(geodesics_repr)[0]
nice_geodesic1

[phi + 1    -phi       0      -1       0       0]
[      0       0       0       0       0       1]
[phi + 1    -phi       0    -phi       0       0]
[      0       0       0       0       1       0]
[    phi    -phi       0       0       0       0]
[      0       0       1       0       0       0]

Here is a cached result to skip the computation:

In [49]:
nice_geodesic1 = matrix(Qphi, 6, 6, [
    [phi + 1,   -phi,      0,     -1,      0,      0],
    [      0,      0,      0,      0,      0,      1],
    [phi + 1,   -phi,      0,   -phi,      0,      0],
    [      0,      0,      0,      0,      1,      0],
    [    phi,   -phi,      0,      0,      0,      0],
    [      0,      0,      1,      0,      0,      0]
], immutable=True)

In [50]:
G_matrix((b*d*a*c*e)^2) == nice_geodesic1

True

Every Long manifold has a nice geodesic:

In [51]:
[G((b*d*a*c*e)^2) in sg for sg in SGs]

[True, True, True, True, True, True, True, True]

In [56]:
def word_search(m, silent=True):
    mid_phi = sum(P^-1*v for v in vertsH).change_ring(RDF)
    mid_phi = mid_phi / sqrt(-mid_phi * Jphi * mid_phi / phi.n())
    def dist(m):
        return acosh(-mid_phi * Jphi * m.change_ring(RDF) * mid_phi)
    dst = dist(m)
    word = []
    end1 = 5
    while True:
        cur = -1
        if not silent:
            print()
            print(dst.n())
        for k,r in enumerate(refl_phi[:end1]):
            mr = m * r
            dst2 = dist(mr)
            if not silent:
                print(k,dst2.n())
            if dst2 is not None and dst2.n() < dst.n():
                dst = dst2
                newm = mr
                cur = k
        if cur != -1:
            word.append(cur)
            m = newm
        elif end1 == 5:
            end1 = 7
        else:
            if not silent:
                print()
            return (m, word[::-1])

## Nice pairs

Two nice geodesics $\gamma_1, \gamma_2$ in $\mathbb H^5$ form a _nice pair_ if their distance is $\cosh^{-1}(3 + \sqrt{5}) = 2.339472\dots$ \
and, if $S$ is the segment realizing their distance and $\pi \colon \mathbb H^5 \to \mathbb H^4$ is an orthogonal projection along $S$, \
then $\pi(\gamma_1), \pi(\gamma_2)$ make an angle of $\cos^{-1}\left[\frac{1}{\sqrt{29}}\sqrt{22-2\sqrt{5}}\right] \sim 38.97^\circ$.

In [52]:
def image_vector_space(m, vs):
    """Returns the image of the vector space vs through the matrix m."""
    return span(m*v for v in vs.basis_matrix())

In [53]:
def midplane_reflection(v1,v2, ring=AA):
    """
    Returns the reflection in the mid-plane of v1, v2.
        - v1, v2 are 1-dimensional vector spaces.
    """
    v1v2orth = ((v1+v2).basis_matrix() * Jphi).right_kernel().basis_matrix()
    V1 = v1.basis_matrix()
    V2 = v2.basis_matrix()
    bas = block_matrix([[V1], [V2], [v1v2orth]], subdivide=False).transpose()
    #print(bas.change_ring(RDF))
    w1 = V1.row(0)
    w2 = V2.row(0)
    ratio = sqrt((w1 * Jphi * w1)/(w2 * Jphi * w2))
    isometry = block_diagonal_matrix([matrix(ring, [[0,1/ratio],[ratio,0]])] + [matrix([[1]])]*4, subdivide=False)
    return bas * isometry * bas^-1

In [54]:
def spectrum_double_projection_nontriv(p1, p2):
    ab = orthogonal_projection(p1.basis_matrix().transpose()) * orthogonal_projection(p2.basis_matrix().transpose())
    return ab.eigenvalues()

In [55]:
def least_distance_point(p1, p2):
    """Returns the point of p1 which realizes the least distance from p2, as a 1-dimensional vector space."""
    ab = double_projection(p1, p2)
    eig = max(max(ab.eigenvalues()), 1)
    return (ab-eig).right_kernel()

Every Long manifold has a nice pair, involving the axes of $(bdace)^2$ and \
$babc(ba)^2cbdcbe(dc(ba)^2cbabc)^2dcedc(ba)^2cbabcdc(ba)^2cbabdcbabcdedc(ba)^2cb$:

In [56]:
print([G(free_word("babcbabacbdcbedcbabacbabcdcbabacbabcdcedcbabacbabcdcbabacbabdcbabcdedcbabacb")) in sg for sg in SGs])

[True, True, True, True, True, True, True, True]


In [57]:
nice_geodesic2 = G_matrix("babcbabacbdcbedcbabacbabcdcbabacbabcdcedcbabacbabcdcbabacbabdcbabcdedcbabacb")

In [58]:
conjugacy_representative(nice_geodesic2) == nice_geodesic1

True

In [59]:
axis1 = axis(nice_geodesic1, lam)
axis2 = axis(nice_geodesic2, lam)

In [60]:
axis1 == axis2

False

In [61]:
cosh_dist_axes = Qphi(cosh_dist_hyperbolic_flats(axis1, axis2))
cosh_dist_axes

2*phi + 2

In [62]:
ldp1 = least_distance_point(axis1, axis2)
ldp2 = least_distance_point(axis2, axis1)

In [63]:
refl_ldp = midplane_reflection(ldp1, ldp2)

In [64]:
axis2_1 = image_vector_space(refl_ldp, axis2)

In [65]:
cos_angle_hyperbolic_flats(axis1, axis2_1).radical_expression()

sqrt(-2/29*sqrt(5) + 22/29)

## Searching for nice pairs

We now show that there is only one $\Gamma$-orbit of nice pairs.\
Let $x$ be a point on a nice geodesic $\gamma$ (axis of $(abcde)^2$) and let $\gamma'$ make a nice pair with $\gamma$. \
Write $\gamma' = h(\gamma)$ for some $h \in \Gamma$. \
Let $y$ be the point of $\gamma$ nearest to $\gamma'$ and let $y'$ be the point of $\gamma'$ nearest to $\gamma$. \
Let $D = d(x, y), D' = d(y', h(x))$. Up to conjugacy (i.e. translating $\gamma'$ with the appropriate conjugates of $bdace \in \Gamma$), \
we may assume $D, D' \le \frac{1}{4} \log(1.849599\dots)$. (The translation distance of $bdace$ is $\frac{1}{2} \log(1.849599\dots)$.)

Now $d(x, h(x))$ is the distance between the endpoints of a chain of $3$-segments (of lengths $D$, $\cosh^{-1}(3 + \sqrt{5})$, $D'$),
meeting at right angles. \
This is bounded by $2\cdot \mathrm{pyth}\left(\frac{1}{4} \log(1.849599\dots), \frac{1}{2} \cosh^{-1}(3 + \sqrt{5})\right)$. \
The bound is not tight as it does not take into account the $38.97^\circ$ angle between the two geodesics.

In [66]:
COSH_MAXD_AXES = cosh(2*acosh(cosh(log(RIF(lam))/4) * cosh(acosh(RIF(cosh_dist_axes))/2)))

In [67]:
COSH_MAXD_AXES, COSH_MAXD_AXES.absolute_diameter()

(5.38463291992559?, 1.24344978758018e-14)

In [81]:
pivot_axis = normalize(axis_segment_endpoints(axis1)[0])
pivot_rif = pivot_axis.change_ring(RIF)

In [75]:
acosh(RIF(cosh_dist(normalize(axis_segment_endpoints(axis1)[0]), normalize(ldp1.gen(0)))))

0.307484487771101?

In [76]:
acosh(RIF(cosh_dist(normalize(axis_segment_endpoints(axis1)[1]), normalize(ldp1.gen(0)))))

0.461226731656651?

In [77]:
acosh(RIF(cosh_dist(normalize(axis_segment_endpoints(axis1)[1]), normalize(axis_segment_endpoints(axis1)[0]))))

0.153742243885550?

In [80]:
3/4 * log(lam_rif)

0.461226731656651?

To avoid memory issues, we pre-filter the conjugators via a distance test, carried out in the real interval field.

In [157]:
def cosh_squared_dist_test(ortho, m, dsq):
    """
    Returns False if dsq cannot represent the squared hyperbolic cosine of the distance between line and m(line).
        - ortho:       orthogonal projection onto a subspace (RIF)
        - m:           isometry (exact)
        - dsq:         value of cosh(dist)^2 to test
    Return type:
        - bool
    """
    ab = ortho * m * ortho * m^-1
    return det(ab.change_ring(RIF) - dsq).contains_zero()

In [83]:
ortho_rif = orthogonal_projection(axis1.basis_matrix().transpose()).change_ring(RIF)

In [84]:
cosh_dist_axes_phi_rif = RIF(cosh_dist_axes * phi)
cosh_dist_axes_squared = cosh_dist_axes^2

In [102]:
%%time
conjugators_filtered = [
    (m0, s) for m0, m1, s in tessellation_search(COSH_MAXD_AXES, pivot=pivot_axis, string=True)
        if not -pivot_rif * Jphi * m1 * pivot_rif < cosh_dist_axes_phi_rif      # pivot_axis is translated by at least acosh(3 + sqrt(5))
        and cosh_squared_dist_test(ortho_rif, m0, cosh_dist_axes_squared)       # distance of axes is compatible with acosh(3 + sqrt(5))
]

2883700
CPU times: user 50min 55s, sys: 26.2 s, total: 51min 21s
Wall time: 51min 32s


In [103]:
len(conjugators_filtered)

334

In [104]:
%%time
conjugators = [(m0, s) for m0, s in conjugators_filtered
                    if RIF(cosh_dist_hyperbolic_flats(axis1, image_vector_space(m0, axis1)) - cosh_dist_axes).contains_zero()]

CPU times: user 4.32 s, sys: 56.3 ms, total: 4.38 s
Wall time: 4.55 s


In [107]:
len(conjugators)

16

All these conjugators produce valid nice pairs:

In [109]:
%%time
assert(all(cosh_dist_hyperbolic_flats(axis1, image_vector_space(m0, axis1)) == cosh_dist_axes for m0, s in conjugators))

CPU times: user 168 ms, sys: 7 μs, total: 168 ms
Wall time: 168 ms


## Nice planes

These conjugators come in pairs $h, h\cdot g$. \
This is to be expected, since nice pairs are contained in the preimage of facet $g$ in $\mathbb H^5$.

In [113]:
sorted(["".join(["abcdefg"[j] for j in s]) for m0, s in conjugators])

['ababacbabadcbabacbabcdcbabacbdcedcbabcde',
 'ababacbabadcbabacbabcdcbabacbdcedcbabcdeg',
 'ababacbabadcbabacbabdcbabacbdcedcbabcde',
 'ababacbabadcbabacbabdcbabacbdcedcbabcdeg',
 'ababcbabadcbabacbabcdcbabacbdcedcbabcde',
 'ababcbabadcbabacbabcdcbabacbdcedcbabcdeg',
 'ababcbabadcbabacbabdcbabacbdcedcbabcde',
 'ababcbabadcbabacbabdcbabacbdcedcbabcdeg',
 'bcbdcbedcbabacbabdcbabaedcbabacbabdcbaba',
 'bcbdcbedcbabacbabdcbabaedcbabacbabdcbabag',
 'bcbdcedcbabacbabdcbabaedcbabacbabdcbaba',
 'bcbdcedcbabacbabdcbabaedcbabacbabdcbabag',
 'cbdcbedcbabacbabdcbabaedcbabacbabdcbaba',
 'cbdcbedcbabacbabdcbabaedcbabacbabdcbabag',
 'cbdcedcbabacbabdcbabaedcbabacbabdcbaba',
 'cbdcedcbabacbabdcbabaedcbabacbabdcbabag']

Every geodesic forming a nice pair with the axis of $(bdace)^2$ is the axis of
$k (bdace)^2 k^{-1}$, \
with $k = (bdace)^n \cdot h \cdot (bdace)^m$ for some $h$ in `conjugators` and some $m,n \in \mathbb Z$.

Without loss of generality, we may assume $h \in G$ (i.e. it does not end with $g$) and $m = 0$. \
Hence, the other geodesic in a nice pair is the axis of an element of $G$. \
Furthermore, any nice pair in $N_{i,I}^\pm$ must be contained in one of the two embedded copies of the Long manifold $Z_i$ or one of the 30 embedded copies of its $17$-fold cover $X_i$.

The latter case may be excluded, since the fundamental group $\pi_1(X_i)$ is contained in $K$ (normal of index $979200$ in $\Gamma$), which does not contain $(bdace)^2$:

In [121]:
actK = libgap.FactorCosetAction(G, G.subgroup([a, c, e, d*e*c*d, b*a*c*b*a*b]))
K = libgap.Kernel(actK)
print(G(b*d*a*c*e)^2 in K)

False


Define a graph $\mathcal G$ as follows. 
The vertices are nice geodesics in $\mathbb H^5$, and two vertices are connected by an edge if they form a nice pair.

We now show that the span of each connected component is a $3$-dimensional subspace of $\mathbb H^5$: a _nice plane_. \
(If we instead consider the induced subgraph $\mathcal G_{i,I}^\pm$ on the lifts of nice geodesics of a fixed $N_{i,I}^\pm$, then the spans of connected components are _contained_ in nice planes.)

To do so, we show that the nice geodesic $(bdace)^2$, and all its neighbors $(bdace)^n h (bdace)^2 h^{-1} (bdace)^{-n}$, \
span a $3$-dimensional subspace of $\mathbb H^5$, i.e. a $4$-dimensional subspace of $\mathbb R^{5,1}$. \
It suffices to find the span $\Pi_3$ of $(bdace)^2$ and all $h (bdace)^2 h^{-1}$, and check that $\Pi_3$ is invariant for $bdace$.

The set of nice planes is the $\Gamma$-orbit of $\Pi_3$.

In [135]:
Pi_3 = (axis1 + sum(axis(m0 * nice_geodesic1 * m0^-1, lam) for m0, s in conjugators)).change_ring(Qphi)
Pi_3

Vector space of degree 6 and dimension 4 over Number Field in phi with defining polynomial x^2 - x - 1 with phi = 1.618033988749895?
Basis matrix:
[  1   0   0   0   1 phi]
[  0   1   0   0   0  -1]
[  0   0   1   0   0  -1]
[  0   0   0   1  -1   0]

In [136]:
image_vector_space(G_matrix(b*d*a*c*e), Pi_3) == Pi_3

True

In [137]:
Pi_3 == (axis1 + axis2).change_ring(Qphi)

True

This provides a simpler description of $\Pi_3$, and also shows that there exists
a connected component of $\mathcal G_{i,I}^\pm$ whose span is a whole nice plane.

## Recovering the $Z_i$

We now search for pairs of nice planes at a distance of $\cosh^{-1} \frac{1}{11}(17 + 4\sqrt{5}) = 1.502888\dots$. \
We show that this distance can only occur between two nice hyperplanes in the same lift of $Z_i$.

Let the facet $g$ be contained in a ball $B(x, r)$. Then we only need to search the tessellation up to \
$2\cdot \mathrm{pyth}\left(r, \frac 12 \cosh^{-1} \frac{1}{11}(17 + 4\sqrt{5})\right).$

In [161]:
cosh_dist_planes = Qphi((17 + 4*sqrt(5))/11)
cosh_dist_planes_squared = cosh_dist_planes^2
cosh_dist_planes.n()

2.35857017363629

A special point on facet $g$, minimizing the distance from its furthest vertex:

In [152]:
mid59 = midpoint(vertices[5], vertices[9])

In [154]:
facet_g_radius = max(cosh_dist(mid59, v) for v in vertices[5:])
facet_g_radius

1.203001910015092?

In [156]:
COSH_MAXD_PLANES = cosh(2 * acosh(
                          cosh(acosh(RIF(cosh_dist_planes))/2) 
                        * RIF(facet_g_radius)
                        ))
COSH_MAXD_PLANES, COSH_MAXD_PLANES.absolute_diameter()

(3.86056841672709?, 8.88178419700125e-15)

In [159]:
ortho_Pi_3_rif = orthogonal_projection(Pi_3.basis_matrix().transpose()).change_ring(RIF)

In [162]:
%%time
isom_nice_planes_all = [(m0, s) for m0, m1, s in tessellation_search(COSH_MAXD_PLANES, pivot=mid59, string=True)
                          if cosh_squared_dist_test(ortho_Pi_3_rif, m1, cosh_dist_planes_squared)]

811200
CPU times: user 21min 33s, sys: 4.53 s, total: 21min 37s
Wall time: 21min 41s


In [179]:
%%time
isom_nice_planes = [(m0, s) for m0, s in isom_nice_planes_all
          if cosh_dist_hyperbolic_flats(Pi_3, image_vector_space(m0, Pi_3)) == cosh_dist_planes]

CPU times: user 7min 41s, sys: 1.28 s, total: 7min 42s
Wall time: 7min 43s


In [181]:
%time multiset(5 in s for m0, s in isom_nice_planes)

CPU times: user 30 ms, sys: 1e+03 ns, total: 30 ms
Wall time: 29.7 ms


{False: 16176}

No isometry in $\Gamma$ sending a nice plane to another contains $f$. \
Hence, it belongs to the stabilizer $\langle a,b,c,d,e,g \rangle$ of the supporting hyperplane of facet $g$. \
Hence, the only pairs of nice planes at a distance $1.502888\dots$ occur in the same lift of $Z_i$.

Such a lift can be recovered from a second nice plane of a Long manifold, obtained using a generator of the fundamental group:

In [188]:
Pi_3x, = {image_vector_space(G_matrix(long_gens[j][k]), Pi_3)
    for j,k in [
        (0,2),
        (1,4),
        (2,3),
        (3,4),
        (4,3),
        (5,4),
        (6,2),
        (7,4)]}
Pi_3x

Vector space of degree 6 and dimension 4 over Number Field in phi with defining polynomial x^2 - x - 1 with phi = 1.618033988749895?
Basis matrix:
[         1          0          0          0 -2*phi - 1  3*phi + 2]
[         0          1          0          0          0         -1]
[         0          0          1          0  2*phi + 2 -2*phi - 3]
[         0          0          0          1         -1          0]

In [190]:
Pi_4 = Pi_3 + Pi_3x
Pi_4

Vector space of degree 6 and dimension 5 over Number Field in phi with defining polynomial x^2 - x - 1 with phi = 1.618033988749895?
Basis matrix:
[      1       0       0       0       0 phi + 1]
[      0       1       0       0       0      -1]
[      0       0       1       0       0      -1]
[      0       0       0       1       0      -1]
[      0       0       0       0       1      -1]

## Five orthogonal lines

In [194]:
Pi_0 = ldp1

In [195]:
Pi_1 = axis1

In [196]:
Pi_2 = axis1 + ldp2

In [199]:
Pi_5 = Qphi^6

We have finally recovered a flag of subspaces $\Pi_0, \Pi_1, \dots, \Pi_4, \Pi_5 = \mathbb H^5$ given a manifold $N_{i,I}^\pm$. \
This is equivalent to an ordered sequence of orthogonal lines $\ell_0, \ell_1, \dots, \ell_4$ through $\Pi_0$,
such that $\Pi_i = \mathrm{span}(\Pi_0, \ell_0, \dots, \ell_{i-1})$.

In [197]:
def orthogonal_subspace(vs):
    return (vs.basis_matrix() * Jphi).right_kernel()

In [216]:
flag = [vs.change_ring(AA) for vs in (Pi_0, Pi_1, Pi_2, Pi_3, Pi_4, Pi_5)]
lines = [(orthogonal_subspace(Pi_i) & Pi_i1) + flag[0] for Pi_i, Pi_i1 in zip(flag[:-1], flag[1:])]

assert(all(sum(lines[:j], flag[0]) == flag[j]) for j in range(6))

The following is a cached definition of `lines`, if one wishes to avoid computing all previous cells:

In [264]:
lines = [span([vector(AA, t) for t in l])
    for l in [[
        (1, 0, 5*lam^7 - 7*lam^5 - 12*lam^4 - 11*lam^3 - 5*lam^2 + 3*lam + 4, 3*lam^7 + lam^6 - 4*lam^5 - 9*lam^4 - 9*lam^3 - 5*lam^2 + 4,
         -3*lam^7 - lam^6 + 4*lam^5 + 9*lam^4 + 9*lam^3 + 5*lam^2 - 3, -3*lam^7 + 4*lam^5 + 7*lam^4 + 7*lam^3 + 4*lam^2 - lam - 2),
        (0, 1, -1, -3*lam^7 + 4*lam^5 + 7*lam^4 + 7*lam^3 + 4*lam^2 - lam - 2, 3*lam^7 - 4*lam^5 - 7*lam^4 - 7*lam^3 - 4*lam^2 + lam + 2, 0)
    ],
    [
        (1, 0, 2*lam^7 - 3*lam^5 - 5*lam^4 - 4*lam^3 - lam^2 + 2*lam + 2, 2, -1, 0),
        (0, 1, -2, -4*lam^7 + 6*lam^5 + 10*lam^4 + 8*lam^3 + 2*lam^2 - 4*lam - 4, 4*lam^7 - 6*lam^5 - 10*lam^4 - 8*lam^3 - 2*lam^2 + 4*lam + 4, 1)
    ],
    [
        (1, 0, -2*lam^7 + 3*lam^5 + 5*lam^4 + 4*lam^3 + lam^2 - 2*lam, -2*lam^6 + 4*lam^4 + 4*lam^3 + 2*lam^2 + 2*lam - 2,
         2*lam^6 - 4*lam^4 - 4*lam^3 - 2*lam^2 - 2*lam + 3, 4*lam^7 - 6*lam^5 - 10*lam^4 - 8*lam^3 - 2*lam^2 + 4*lam + 2),
        (0, 1, -2*lam^6 + 4*lam^4 + 4*lam^3 + 2*lam^2 + 2*lam - 2, -10*lam^7 + 14*lam^5 + 24*lam^4 + 22*lam^3 + 10*lam^2 - 6*lam - 6,
         10*lam^7 - 14*lam^5 - 24*lam^4 - 22*lam^3 - 10*lam^2 + 6*lam + 6, 2*lam^6 - 4*lam^4 - 4*lam^3 - 2*lam^2 - 2*lam + 1)
    ],
    [
        (1, 0, 3*lam^7 + lam^6 - 4*lam^5 - 9*lam^4 - 9*lam^3 - 5*lam^2 + 4, 5*lam^7 - 7*lam^5 - 12*lam^4 - 11*lam^3 - 5*lam^2 + 3*lam + 4,
         -6*lam^7 - lam^6 + 8*lam^5 + 16*lam^4 + 16*lam^3 + 9*lam^2 - lam - 5, 0),
        (0, 1, 2*lam^7 - lam^6 - 3*lam^5 - 3*lam^4 - 2*lam^3 + 3*lam, -5*lam^7 + lam^6 + 7*lam^5 + 10*lam^4 + 9*lam^3 + 4*lam^2 - 4*lam - 3,
         3*lam^7 - 4*lam^5 - 7*lam^4 - 7*lam^3 - 4*lam^2 + lam + 1, 1)
    ],
    [
        (1, 0, 3*lam^7 - lam^6 - 4*lam^5 - 5*lam^4 - 5*lam^3 - 3*lam^2 + 2*lam + 2, lam^7 - lam^5 - 2*lam^4 - 3*lam^3 - 3*lam^2 - lam + 2,
         -lam^6 + 2*lam^4 + 2*lam^3 + lam^2 + lam - 1, 0),
        (0, 1, -2*lam^7 - lam^6 + 3*lam^5 + 7*lam^4 + 6*lam^3 + 2*lam^2 - lam - 2, -5*lam^7 - lam^6 + 7*lam^5 + 14*lam^4 + 13*lam^3 + 6*lam^2 - 2*lam - 3,
         3*lam^7 - 4*lam^5 - 7*lam^4 - 7*lam^3 - 4*lam^2 + lam + 3, 1)
    ]]
]

The configuration $\ell_1, \ell_2, \dots, \ell_5$ can only occur in a certain, fixed position with respect to the tessellation. \
Hence, given such a configuration in $\mathbb H^5$ extracted from $N_{i,I}^\pm$,
the tessellation is determined up to the group $R$ of $32$ isometries that flip one or more $\ell_i$. \
Let $r_i \equiv$ `refl_lines[i]` be the reflection that fixes $\ell_j$ pointwise for $j \ne i$ and flips $\ell_i$;
then $R$ is the abelian $2$-group generated by $r_0, r_1, \dots, r_4$.

In [217]:
refl_lines = [-1 + 2*orthogonal_projection((orthogonal_subspace(ln)+flag[0]).basis_matrix().transpose()) for ln in lines]

Any tessellation of $N_{i,I}^\pm$ is determined as the projection of $r(T)$ to $N_{i,I}^\pm$,
where $T$ is the canonical tessellation of $\mathbb H^5$ and $r \in R$. \
Clearly, $r(T)$ depends only on the left coset $r \Gamma$.

In order to prove that the tessellation is unique, we need to exclude cases where $r \not \in \Gamma$, \
by showing that in such cases $r(T)$ does not project to a valid tessellation on $N_{i,I}^\pm$.

In [229]:
refl_lines_all = {t: product(refl_lines[j] for j in range(5) if t[j])*identity_matrix(AA,6) for t in itertools.product((0,1), repeat = 5)}

In [243]:
for t, r in refl_lines_all.items():
    try:
        rphi = r.change_ring(Qphi)
        name = "id" if max(t) == 0 else "*".join(f"r{j}" for j,s in enumerate(t) if s)
        print(name, "has entries in Qphi:")
        print(rphi)
        print()
    except TypeError as ex:
        pass

id has entries in Qphi:
[1 0 0 0 0 0]
[0 1 0 0 0 0]
[0 0 1 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 1 0]
[0 0 0 0 0 1]

r4 has entries in Qphi:
[ 5*phi + 4 -2*phi - 1 -2*phi - 1 -2*phi - 1 -2*phi - 1 -2*phi - 1]
[ 3*phi + 2       -phi   -phi - 1   -phi - 1   -phi - 1   -phi - 1]
[ 3*phi + 2   -phi - 1       -phi   -phi - 1   -phi - 1   -phi - 1]
[ 3*phi + 2   -phi - 1   -phi - 1       -phi   -phi - 1   -phi - 1]
[ 3*phi + 2   -phi - 1   -phi - 1   -phi - 1       -phi   -phi - 1]
[ 3*phi + 2   -phi - 1   -phi - 1   -phi - 1   -phi - 1       -phi]

r3 has entries in Qphi:
[ 5/11*phi + 4/11  2/11*phi - 5/11  2/11*phi - 5/11 -2/11*phi + 5/11 -2/11*phi + 5/11  2/11*phi - 5/11]
[ 3/11*phi - 2/11 -1/11*phi + 8/11 -1/11*phi - 3/11  1/11*phi + 3/11  1/11*phi + 3/11 -1/11*phi - 3/11]
[ 3/11*phi - 2/11 -1/11*phi - 3/11 -1/11*phi + 8/11  1/11*phi + 3/11  1/11*phi + 3/11 -1/11*phi - 3/11]
[-3/11*phi + 2/11  1/11*phi + 3/11  1/11*phi + 3/11 -1/11*phi + 8/11 -1/11*phi - 3/11  1/11*phi + 3/11]
[-3/11*phi + 2/11 

Here we see that $r_0 r_2$ and $r_4$ are in $\mathrm O(J_\varphi, \mathbb Z[\varphi]) = \Gamma$. \
Hence, without loss of generality, it suffices to exclude the reflections $r_0, r_1, r_3, r_0 r_1, r_0 r_3, r_1 r_3, r_0 r_1 r_3$.

In [240]:
remaining_tuples = [tuple(v.change_ring(ZZ)) for v in span([(GF(2)^5).gen(j) for j in (0,1,3)]) if v]

In [241]:
remaining_tuples

[(1, 0, 0, 0, 0),
 (0, 1, 0, 0, 0),
 (1, 1, 0, 0, 0),
 (0, 0, 0, 1, 0),
 (1, 0, 0, 1, 0),
 (0, 1, 0, 1, 0),
 (1, 1, 0, 1, 0)]

In [245]:
for t in remaining_tuples:
    r = refl_lines_all[t]
    name = "id" if max(t) == 0 else "*".join(f"r{j}" for j,s in enumerate(t) if s)
    try:
        mat = image_vector_space(r, Pi_3x).basis_matrix()
        mat_phi = mat.change_ring(Qphi)
        print(f"({name:>14})(Pi_3x) is     defined over Q[phi]")
    except TypeError as ex:
        print(f"({name:>14})(Pi_3x) is NOT defined over Q[phi]")

(            r0)(Pi_3x) is NOT defined over Q[phi]
(            r1)(Pi_3x) is NOT defined over Q[phi]
(         r0*r1)(Pi_3x) is NOT defined over Q[phi]
(            r3)(Pi_3x) is     defined over Q[phi]
(         r0*r3)(Pi_3x) is NOT defined over Q[phi]
(         r1*r3)(Pi_3x) is NOT defined over Q[phi]
(      r0*r1*r3)(Pi_3x) is NOT defined over Q[phi]


All reflections in $\{r_0, r_1, r_3, r_0 r_1, r_0 r_3, r_1 r_3, r_0 r_1 r_3\}$ except $r_3$ 
send a nice plane (present in all $N_{i,I}^\pm$) \
to a non-nice plane (because all nice planes are defined over $\mathbb Q(\varphi)$). \
This is not admissible, so it only remains to exclude $r_3$.

In [249]:
r3 = refl_lines[3].change_ring(Qphi)
r3

[ 5/11*phi + 4/11  2/11*phi - 5/11  2/11*phi - 5/11 -2/11*phi + 5/11 -2/11*phi + 5/11  2/11*phi - 5/11]
[ 3/11*phi - 2/11 -1/11*phi + 8/11 -1/11*phi - 3/11  1/11*phi + 3/11  1/11*phi + 3/11 -1/11*phi - 3/11]
[ 3/11*phi - 2/11 -1/11*phi - 3/11 -1/11*phi + 8/11  1/11*phi + 3/11  1/11*phi + 3/11 -1/11*phi - 3/11]
[-3/11*phi + 2/11  1/11*phi + 3/11  1/11*phi + 3/11 -1/11*phi + 8/11 -1/11*phi - 3/11  1/11*phi + 3/11]
[-3/11*phi + 2/11  1/11*phi + 3/11  1/11*phi + 3/11 -1/11*phi - 3/11 -1/11*phi + 8/11  1/11*phi + 3/11]
[ 3/11*phi - 2/11 -1/11*phi - 3/11 -1/11*phi - 3/11  1/11*phi + 3/11  1/11*phi + 3/11 -1/11*phi + 8/11]

In [254]:
r3.denominator()

11

The matrix $r_3$ is in $\frac{1}{11} \mathrm{M}(6, \mathbb Z[\varphi])$.
If $r_3(T)$ were a valid tessellation, then for any $h \in H_i' \coloneqq \pi_1(Z_i)$, we would have $r_3 h r_3^{-1} = r_3 h r_3 \in \Gamma$.

_A priori_, we only have $r_3 h r_3 \in \frac{1}{121} \mathrm{M}(6, \mathbb Z[\varphi])$. Indeed, we can show that for each $i$ there exists $h \in H_i' < \pi_1(N_{i,I}^\pm)$ such that the denominator of $r_3 h r_3$ is not $1$.

In [256]:
for long_gens_i in long_gens:
    print([(r3 * G_matrix(gen) * r3^-1).denominator() for gen in long_gens_i])

[1, 121, 121, 121, 1, 121, 1, 121]
[1, 1, 121, 121, 121, 121, 121, 121]
[1, 1, 121, 121, 1, 121, 121]
[1, 121, 121, 1, 121, 121]
[1, 1, 121, 121, 121, 1, 121, 121, 121]
[1, 1, 121, 121, 121, 121, 121]
[1, 121, 121, 1, 1, 121, 121]
[1, 1, 121, 121, 121, 121]


Indeed, we can check this by only looking at the generators, finding that some conjugates have denominator $121$ and hence are not in $\Gamma$. \
This excludes $r_3$ and completes the proof of:

***Theorem***. The manifolds $N_{i,I}^\pm$ are uniquely tessellated by copies of $P$.